In [ ]:
from docGPT_core import *

In [ ]:
set_tokens(OPENAI_TOKEN='',
                  HF_TOKEN="")
hf_embeddings = Embedding(model_name='sentence-transformers/all-MiniLM-L6-v2', model_type = 'hf')
openai_embeddings = Embedding(model_type='openai')
persona = Persona(personality_type='explainer')
llm = Llm(model_type='gpt-3.5-turbo')

In [ ]:
my_loader = DirectoryLoader('data')
docs = my_loader.load()
text_split = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=1000)

In [ ]:
splits = text_split.split_documents(docs)

In [ ]:
llm.model.get_num_tokens(splits[1100].page_content)

In [ ]:
import time
vs = FAISS.from_documents([splits[0]], embedding=openai_embeddings.model)
batch_size = 20
num_batches = len(splits) // batch_size + 1
for i in range(num_batches):
    start_idx = (i * batch_size)
    end_idx = min(start_idx + batch_size, len(splits))
    batch = splits[start_idx:end_idx]
    vs.add_documents(batch)
    print(f"Added {len(batch)} documents to vectorstore. Progress: {end_idx}/{len(splits)}")
    time.sleep(0.5)

In [ ]:
Vs = VectorStore(embedding_model=openai_embeddings, from_large_embeddings=True, vectorstore=vs)
Vs.save('store/medqa.pkl')

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm.model, chain_type='stuff', retriever=Vs.store.as_retriever(search_kwargs={"k": 3}), verbose=True, chain_type_kwargs=persona.persona)

In [ ]:
q = """
A 31-year-old G2P2 female at 40 weeks gestation presents to the hospital following a rush of water that came from her vagina. She is 4 cm dilated and 80% effaced. Fetal heart tracing shows a pulse of 155/min with variable decelerations. About 12 hours after presentation, she gives birth to a 6 lb 15 oz baby boy with APGAR scores of 8 and 9 at 1 and 5 minutes, respectively. Which of the following structures is responsible for inhibition of female internal genitalia?

"A": "Spermatogonia", "B": "Allantois", "C": "Syncytiotrophoblast", "D": "Sertoli cells", "E": "Leydig cells"

"""

res = qa(inputs={"query": q})
print(res['result'])

In [ ]:
res = qa(inputs={"query": q})

In [ ]:
print(res['result'])